In [19]:
import numpy as np
import pandas as pd


class gm1n(object):
    """
    定义GM(1,N)模型

    使用方法：
    1.实例化类     model = gm1n(rel_data, sys_data, predict_step=2, discrete=False, background_coff=0.5)
    2.训练模型     model.fit()
    3.预测        model.predict()
    4.计算误差    model.loss()

    输入数据：
    - rel_data: 相关因素序列，Pandas DataFrame，多列数据
    - sys_data: 系统行为序列，Pandas DataFrame，单列数据
    - predict_step: 预测步长，默认为2
    - discrete: 是否使用离散模型，默认为False（使用连续模型）
    - background_coff: 背景值系数，默认为0.5
    """

    def __init__(self, rel_data: pd.DataFrame, sys_data: pd.DataFrame, predict_step: int = 2,
                 discrete: bool = False, background_coff: float = 0.5):
        """
        初始化GM(1,N)模型

        参数：
        - rel_data: 相关因素序列，Pandas DataFrame
        - sys_data: 系统行为序列，Pandas DataFrame
        - predict_step: 预测步长，默认为2
        - discrete: 是否使用离散模型，默认为False（使用连续模型）
        - background_coff: 背景值系数，默认为0.5
        """
        self.sys_data = sys_data  # 系统行为序列
        self.rel_data = rel_data  # 相关因素序列
        self.data_shape = self.sys_data.shape[0] - predict_step  # 数据长度减去预测步长
        self.predict_step = predict_step  # 预测步长
        self.discrete = discrete  # 是否使用离散模型
        self.bgc = background_coff  # 背景值系数
        self.coff = None  # 模型参数
        self.sim_data = [self.sys_data[0]]  # 拟合值数组，初始值为系统行为序列的第一个值
        self.pred_data = []  # 预测值数组

    def least_squares_method_function(self):
        """
        最小二乘法求解模型参数

        功能：
        - 构造矩阵Y和B
        - 使用最小二乘法求解模型参数

        参数：
        - 无

        返回：
        - 无
        """
        # 定义矩阵Y
        Y = self.sys_data.iloc[1:self.data_shape].values.reshape((self.data_shape - 1, 1))
        # 计算背景值
        cum_sys_data = np.cumsum(self.sys_data)
        Z = np.zeros((self.data_shape - 1, 1))
        for i in range(self.data_shape - 1):
            Z[i] = self.bgc * cum_sys_data.iloc[i] + (1 - self.bgc) * cum_sys_data.iloc[i + 1]
        # 计算相关序列的累加
        rel_data_cum = np.cumsum(self.rel_data[:-self.predict_step], axis=0)
        rel_data_cum = rel_data_cum.iloc[1:self.data_shape, :].values
        # 得到矩阵B
        B = np.column_stack((-Z, rel_data_cum))
        # 使用最小二乘求解系数
        self.coff = np.matmul(np.linalg.inv(np.matmul(B.T, B)), np.matmul(B.T, Y))

    def fit(self):
        """
        模型拟合方法

        功能：
        - 调用最小二乘法求解模型参数
        - 根据模型参数计算拟合值

        参数：
        - 无

        返回：
        - 拟合值数组
        """
        self.least_squares_method_function()
        sys_data = self.sys_data.copy().iloc[:self.data_shape]
        sys_data_cum = np.cumsum(sys_data).values
        rel_data_cum = np.cumsum(self.rel_data, axis=0)
        rel_data_cum = rel_data_cum.values
        # 离散的情况
        if self.discrete:
            for i in range(1, self.data_shape + self.predict_step):
                x = 0
                for j in range(rel_data_cum.shape[1]):
                    x += (self.coff[j + 1] / (1 + 0.5 * self.coff[0])) * rel_data_cum[i, j]
                y = x - (self.coff[0] / (1 + 0.5 * self.coff[0])) * sys_data_cum[i]
                if i >= self.data_shape - 1:
                    sys_data = np.append(sys_data, y[0])
                    sys_data_cum = np.cumsum(sys_data)
                self.sim_data.append(y[0])
        # 连续的情况
        else:
            temp_lt = [sys_data_cum[0]]
            for i in range(1, self.data_shape + self.predict_step):
                x = 0
                z = 0
                for j in range(rel_data_cum.shape[1]):
                    x += self.coff[j + 1] * rel_data_cum[i, j]
                    z += rel_data_cum[i, j]
                y = (sys_data_cum[0] - (1 / self.coff[0]) * x) * np.exp(-self.coff[0] * i) + (1 / self.coff[0]) * x
                temp_lt.append(y[0])
            for i in range(len(temp_lt) - 1):
                x = temp_lt[i + 1] - temp_lt[i]
                self.sim_data.append(x)
        return self.sim_data[:-self.predict_step]

    def predict(self):
        """
        模型预测方法

        功能：
        - 根据拟合参数计算预测值

        参数：
        - 无

        返回：
        - 预测值数组
        """
        return self.sim_data[-self.predict_step:]

    def loss(self):
        """
        计算误差方法

        功能：
        - 计算模型误差（待实现）

        参数：
        - 无

        返回：
        - 无
        """
        pass


if __name__ == '__main__':
    """
    主程序入口

    功能：
    - 读取数据
    - 实例化模型
    - 训练模型并输出拟合值
    - 预测并输出预测值
    """
    data = pd.read_excel('GM1N示例数据.xlsx', sheet_name='Sheet3', header=None)
    system_data = data.iloc[:, 0]  # 系统行为序列
    relevent_data = data.iloc[:, 1:]  # 相关因素序列

    model = gm1n(relevent_data, system_data, predict_step=3, discrete=True)
    fit_values = model.fit()  # 拟合值
    print('拟合值为：')
    print(fit_values)
    predict_values = model.predict()  # 预测值
    print('预测值为：')
    print(predict_values)    

拟合值为：
[20556.0, 8288.479987319675, 11105.662939219459, 13364.70443357066, 15010.55616216264, 18024.062450851052]
预测值为：
[51233.43876822575, 73618.88341107947, 92433.72966214709]
